In [74]:
from __future__ import print_function
import argparse
import pandas as pd
import numpy as np
import rosbag
import cv2
import os
from scipy.interpolate import interp1d
from scipy import array
from scipy import interp
from cv_bridge import CvBridge              # , CvBridgeError

In [58]:
#Setting up the parameters
dataset_images="/home/aiwagan/data/udacity-dataset_sensor_camera_center_2016-10-09-03-47-05_0.bag"
dataset_vehicle="/home/aiwagan/data/udacity-dataset_io_vehicle_2016-10-09-03-47-05_0.bag"

skip=0
outdir="/home/aiwagan/output/"
fmt = 'jpg'

In [45]:
startsec = 0

f_steer = open('steering_angle.csv', 'w')
f_gps = open('gps.csv', 'w')

angle_steers = 0
topics_req = ['/vehicle/steering_report', '/vehicle/gps/fix']
print ("reading rosbag ", dataset_vehicle)
bag = rosbag.Bag(dataset_vehicle, 'r')

for topic, msg, t in bag.read_messages(topics=topics_req):
    if startsec == 0:
        startsec = t.to_sec()
        if skip < 24 * 60 * 60:
            skipping = t.to_sec() + skip
            print ("skipping ", skip, " seconds from ", startsec, " to ", skipping, " ...")
        else:
            skipping = skip
            print ("skipping to ", skip, " from ", startsec, " ...")
    else:
        if t.to_sec() > skipping:
            if topic in ['/vehicle/steering_report']:
                f_steer.write(str(msg.header.stamp) + "," + str(msg.steering_wheel_angle)+"\n")

            if topic in ['/vehicle/gps/fix']:
                f_gps.write(str(msg.header.stamp) + "," + str( msg.latitude) + "," + str(msg.longitude)+"\n")

f_gps.close()
f_steer.close()

reading rosbag  /home/aiwagan/data/udacity-dataset_io_vehicle_2016-10-09-03-47-05_0.bag
skipping  0  seconds from  1476010038.81  to  1476010038.81  ...


In [75]:
df_steer = pd.read_csv('steering_angle.csv')
df_gps = pd.read_csv('gps.csv')


def extrap1d(interpolator):
    xs = interpolator.x
    ys = interpolator.y

    def pointwise(x):
        if x < xs[0]:
            return ys[0]+(x-xs[0])*(ys[1]-ys[0])/(xs[1]-xs[0])
        elif x > xs[-1]:
            return ys[-1]+(x-xs[-1])*(ys[-1]-ys[-2])/(xs[-1]-xs[-2])
        else:
            return interpolator(x)

    def ufunclike(xs):
        return array(map(pointwise, array(xs)))

    return ufunclike

func_steer_interp=extrap1d(interp1d(df_steer.ix[:,0],df_steer.ix[:,1]) )
func_gps_latitude_interp=extrap1d(interp1d(df_gps.ix[:,0],df_gps.ix[:,1]) )
func_gps_longitude_interp=extrap1d(interp1d( df_gps.ix[:,0],df_gps.ix[:,2]) )

In [79]:
print(func_steer_interp([1475520790481041481]))

[-0.36826447]


In [ ]:
imgbag = rosbag.Bag(dataset_images, 'r')

f_steer = open('/home/aiwagan/challenge2/challenge2.csv', 'w')
f_gps_long = open('/home/aiwagan/challenge2/challenge3_long.csv', 'w')
f_gps_lati = open('/home/aiwagan/challenge2/challenge3_lat.csv', 'w')


startsec = 0
bridge = CvBridge()

for topic, msg, t in imgbag.read_messages(topics=['/center_camera/image_color/compressed']):
    if startsec == 0:
        startsec = t.to_sec()
        if skip < 24 * 60 * 60:
            skipping = t.to_sec() + skip
            print ("skipping ", skip, " seconds from ", startsec, " to ", skipping, " ...")
        else:
            skipping = skip
            print ("skipping to ", skip, " from ", startsec, " ...")
    else:
        if t.to_sec() > skipping:
            try:
                if topic in ['/center_camera/image_color/compressed']:
                    f_steer.write( str(msg.header.stamp) + ".jpg " + str(func_steer_interp([int(str(msg.header.stamp))])[0]) + "\n" )
                    f_gps_lati.write( str(msg.header.stamp) + ".jpg " + str(func_gps_latitude_interp([int(str(msg.header.stamp))])[0]) + "\n")         
                    f_gps_long.write( str(msg.header.stamp) + ".jpg " + str(func_gps_longitude_interp([int(str(msg.header.stamp))])[0]) + "\n")
                    #bridge.imgmsg_to_cv2(msg.data, "bgr8")
                    #cv_image = msg.data
                    np_arr = np.fromstring(msg.data, np.uint8)
                    cv_image = cv2.imdecode(np_arr, cv2.CV_LOAD_IMAGE_COLOR)
                    cv_image = cv2.resize(cv_image, (160, 120), interpolation=cv2.INTER_CUBIC)
                    image_filename = os.path.join(outdir, str(msg.header.stamp) + '.' + fmt)
                    cv2.imwrite(image_filename, cv_image)
            except:
                print("Error saving image.")



f_steer.close()
f_gps_lati.close()
f_gps_long.close()

In [42]:
print(f(df_steer.ix[400,0]))

-0.368264466524
